# Reto Coppel


## Importación de librerías

In [75]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lit,isnan, when, count, sum, countDistinct, year, month, day

## Conexión al servidor

In [76]:
spark = SparkSession.builder.master("local[*]").getOrCreate()

## Carga de datos

In [77]:
df = spark.read.csv("../dataset.csv", header=True, inferSchema=True)
for col_name in df.columns:
    df = df.withColumnRenamed(col_name, col_name.capitalize())

## Inspección

In [78]:
df.show(5)

+------+----------+--------+------------------+------------------+------------------+------+--------+--------+----------------+
|   _c0|     Fecha|Segmento|      Hora_llegada|      Hora_llamado|       Hora_salida|  Caja|  Tienda|  Status|          Estado|
+------+----------+--------+------------------+------------------+------------------+------+--------+--------+----------------+
|821243|2023-01-01|  retail|0.5034652777777778|0.5034768518518519|0.5035810185185186|caja_a|Tienda_A|Atendido|Ciudad de México|
|821244|2023-01-01|  retail|0.5035810185185186|0.5036157407407408|0.5111736111111111|caja_a|Tienda_A|Atendido|Ciudad de México|
|821245|2023-01-01|  retail|0.5037893518518518| 0.503800925925926|0.5051203703703704|caja_b|Tienda_A|Atendido|Ciudad de México|
|821246|2023-01-01|  retail|0.5038472222222222|0.5051319444444444|0.5093333333333333|caja_b|Tienda_A|Atendido|Ciudad de México|
|821247|2023-01-01|  retail|0.5038935185185185|0.5080717592592593|0.5099814814814815|caja_c|Tienda_A|Ate

In [79]:
print((df.count(), len(df.columns)))

(18510772, 10)


### Tipos de datos

In [80]:
df.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- Fecha: date (nullable = true)
 |-- Segmento: string (nullable = true)
 |-- Hora_llegada: double (nullable = true)
 |-- Hora_llamado: double (nullable = true)
 |-- Hora_salida: double (nullable = true)
 |-- Caja: string (nullable = true)
 |-- Tienda: string (nullable = true)
 |-- Status: string (nullable = true)
 |-- Estado: string (nullable = true)



El tipo de dato de cada atributo corresponde a la naturaleza del atributo.

### Valores nulos

In [81]:
df.agg(*[sum(col(column).isNull().cast("int")).alias(column) for column in df.columns]).show()

+---+-----+--------+------------+------------+-----------+----+------+------+------+
|_c0|Fecha|Segmento|Hora_llegada|Hora_llamado|Hora_salida|Caja|Tienda|Status|Estado|
+---+-----+--------+------------+------------+-----------+----+------+------+------+
|  0|    0|       0|           0|           0|          0|   0|     0|     0|     0|
+---+-----+--------+------------+------------+-----------+----+------+------+------+



No hay valores nulos.

### Valores únicos

In [82]:
df.select(col("Segmento")).distinct().show()

+----------+
|  Segmento|
+----------+
|     banco|
|    retail|
|afiliacion|
+----------+



Hay tres tipos de segmento que corresponden a los siguientes tipos: \
Banco: Bancoppel \
Retail: Coppel \
Afiliación: Promotoría

In [83]:
df.select(col("Caja")).distinct().show(df.count())

+------------+
|        Caja|
+------------+
|      caja_j|
|         p_d|
|      caja_p|
|      caja_f|
|         p_h|
|      caja_l|
|ventanilla_c|
|ventanilla_g|
|         p_l|
|      caja_q|
|         p_p|
|         p_m|
|      caja_n|
|         p_k|
|      caja_i|
|         p_e|
|      caja_b|
|         p_g|
|         p_b|
|ventanilla_b|
|         p_f|
|         p_a|
|ventanilla_a|
|         p_o|
|      caja_d|
|      caja_e|
|ventanilla_e|
|         p_j|
|      caja_k|
|         p_n|
|      caja_a|
|         p_c|
|      caja_m|
|      caja_o|
|      caja_h|
|ventanilla_f|
|      caja_c|
|ventanilla_d|
|         p_i|
|         a_a|
|         a_b|
|         p_r|
|         p_q|
|      caja_r|
|      caja_t|
|         p_s|
|      caja_s|
|ventanilla_i|
|ventanilla_n|
|ventanilla_m|
|         p_v|
|         p_y|
|ventanilla_l|
|      Caja_d|
|      caja_v|
|         p_u|
|      Caja_f|
|      Caja_b|
|         p_z|
|         P_a|
|         p_t|
|ventanilla_h|
|ventanilla_o|
|      Caj

Se tienen 77 cajas distintas que se dividen en tres tipos: caja, ventanilla, p. Hay errores de etiquetado.

In [84]:
df.groupBy(["Segmento", "Caja"]).agg(countDistinct("Caja").alias("Cajas_distintas")).show(df.count())

+----------+------------+---------------+
|  Segmento|        Caja|Cajas_distintas|
+----------+------------+---------------+
|     banco|ventanilla_b|              1|
|    retail|      caja_k|              1|
|    retail|      caja_a|              1|
|     banco|      caja_l|              1|
|afiliacion|         p_a|              1|
|     banco|      caja_k|              1|
|     banco|         p_f|              1|
|afiliacion|ventanilla_f|              1|
|     banco|      caja_d|              1|
|    retail|ventanilla_a|              1|
|afiliacion|      caja_n|              1|
|afiliacion|      caja_a|              1|
|afiliacion|         p_o|              1|
|    retail|      caja_d|              1|
|afiliacion|         p_c|              1|
|    retail|         p_k|              1|
|     banco|      caja_b|              1|
|    retail|      caja_i|              1|
|     banco|      caja_o|              1|
|afiliacion|      caja_h|              1|
|     banco|         p_b|         

Cada segmento tiene diferentes cajas.

In [85]:
df.groupBy("Segmento").agg(countDistinct("Caja").alias("Cajas_distintas")).show()

+----------+---------------+
|  Segmento|Cajas_distintas|
+----------+---------------+
|     banco|             75|
|    retail|             74|
|afiliacion|             62|
+----------+---------------+



In [86]:
df.select(col("Tienda")).distinct().show(df.count())

+---------+
|   Tienda|
+---------+
| Tienda_G|
|Tienda_AE|
|Tienda_AI|
|Tienda_AD|
|Tienda_AH|
| Tienda_M|
| Tienda_U|
| Tienda_L|
|Tienda_AX|
|Tienda_BD|
|Tienda_AG|
| Tienda_D|
|Tienda_AB|
|Tienda_AK|
|Tienda_AF|
|Tienda_AJ|
|Tienda_AM|
| Tienda_P|
| Tienda_R|
| Tienda_V|
| Tienda_O|
| Tienda_I|
| Tienda_N|
| Tienda_A|
| Tienda_B|
|Tienda_BB|
|Tienda_AA|
| Tienda_S|
|Tienda_AN|
|Tienda_AP|
| Tienda_X|
| Tienda_Z|
| Tienda_E|
|Tienda_AO|
|Tienda_BE|
|Tienda_AY|
| Tienda_T|
|Tienda_AC|
|Tienda_BC|
| Tienda_J|
|Tienda_AL|
| Tienda_Q|
| Tienda_H|
|Tienda_BA|
|Tienda_AZ|
| Tienda_Y|
| Tienda_F|
| Tienda_K|
|Tienda_AQ|
| Tienda_C|
+---------+



Se tienen 50 tiendas distintas.

In [52]:
df.select(col("Status")).distinct().show()

+--------+
|  Status|
+--------+
| Ausente|
|Atendido|
+--------+



In [87]:
df.select(col("Estado")).distinct().show(df.count())

+--------------------+
|              Estado|
+--------------------+
|    Ciudad de México|
|             Sinaloa|
| Michoacán de Ocampo|
|             Durango|
|            Guerrero|
|             Chiapas|
|          Guanajuato|
|Coahuila de Zaragoza|
|           Chihuahua|
|             Morelos|
|Veracruz de Ignac...|
|             Yucatán|
|             Tabasco|
|              Oaxaca|
| Baja California Sur|
|              Puebla|
|          Nuevo León|
|           Zacatecas|
|           Querétaro|
|        Quintana Roo|
|              Sonora|
|              México|
|     Baja California|
+--------------------+



Se incluyen solo 23 estados de la república.

Hay errores de etiquetado en el atributo *Caja*.

## Limpieza de datos

Corregir errores de etiquetado y elimar observaciones donde hora de llegada es mayor que la hora de llamado o salida y las observaciones donde su hora de llamado es mayor al horario de salida.

## Transformación de datos

In [69]:
df = df.withColumn("Anio", year(col("Fecha")))
df = df.withColumn("Mes", month(col("Fecha")))
df = df.withColumn("Dia", day(col("Fecha")))
df.show(5)

+------+----------+--------+------------------+------------------+------------------+------+--------+--------+----------------+----+---+---+
|   _c0|     Fecha|Segmento|      Hora_llegada|      Hora_llamado|       Hora_salida|  Caja|  Tienda|  Status|          Estado|Anio|Mes|Dia|
+------+----------+--------+------------------+------------------+------------------+------+--------+--------+----------------+----+---+---+
|821243|2023-01-01|  retail|0.5034652777777778|0.5034768518518519|0.5035810185185186|caja_a|Tienda_A|Atendido|Ciudad de México|2023|  1|  1|
|821244|2023-01-01|  retail|0.5035810185185186|0.5036157407407408|0.5111736111111111|caja_a|Tienda_A|Atendido|Ciudad de México|2023|  1|  1|
|821245|2023-01-01|  retail|0.5037893518518518| 0.503800925925926|0.5051203703703704|caja_b|Tienda_A|Atendido|Ciudad de México|2023|  1|  1|
|821246|2023-01-01|  retail|0.5038472222222222|0.5051319444444444|0.5093333333333333|caja_b|Tienda_A|Atendido|Ciudad de México|2023|  1|  1|
|821247|2023-

## Inspección de las fechas

In [70]:
df.select(col("Anio")).distinct().show()

+----+
|Anio|
+----+
|2023|
|2024|
+----+



In [71]:
df.select(col("Mes")).distinct().show()

+---+
|Mes|
+---+
|  1|
|  2|
|  3|
|  4|
|  5|
|  6|
|  7|
|  8|
|  9|
| 10|
| 11|
| 12|
+---+



In [74]:
df.select(col("Dia")).distinct().orderBy(col("Dia")).show(df.count())

+---+
|Dia|
+---+
|  1|
|  2|
|  3|
|  4|
|  5|
|  6|
|  7|
|  8|
|  9|
| 10|
| 11|
| 12|
| 13|
| 14|
| 15|
| 16|
| 17|
| 18|
| 19|
| 20|
| 21|
| 22|
| 23|
| 24|
| 25|
| 26|
| 27|
| 28|
| 29|
| 30|
| 31|
+---+

